In [1]:
pip install watchdog

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install schedule

In [ ]:
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
import os
import re
import pandas as pd
import datetime

def process_data(df, date_report):
    # Khởi tạo danh sách để lưu trữ các dòng dữ liệu
    data = []

    # Xử lý từng dòng dữ liệu
    cap1 = cap2 = cap3 = cap4 = ""
    unit = day = month = year = note = kpi = ""
    for index, row in df.iterrows():
        line = row['NỘI DUNG']
        unit = row['ĐƠN VỊ'] if not pd.isna(row['ĐƠN VỊ']) else ""
        day = row['NGÀY'] if not pd.isna(row['NGÀY']) else ""
        month = row['THÁNG'] if not pd.isna(row['THÁNG']) else ""
        year = row['NĂM'] if not pd.isna(row['NĂM']) else ""
        note = row['GHI CHÚ'] if not pd.isna(row['GHI CHÚ']) else ""
        kpi = row['CÔNG SUẤT THỰC TẾ (theo tích lũy năm)'] if not pd.isna(row['CÔNG SUẤT THỰC TẾ (theo tích lũy năm)']) else ""
    
        # Xác định cấp của dòng dựa trên các quy tắc logic
        
        if not pd.isna(line) and isinstance(line, str):
            # Xác định cấp của dòng dựa trên các quy tắc logic
            if re.match(r"^[A-Z]+\.\s", line):
                cap1 = line
                cap2 = cap3 = cap4 = ""
            elif re.match(r"^\d+\.\s", line):
                cap2 = line
                cap3 = cap4 = ""
            elif re.match(r"^\d+\.\d+\s", line):
                cap3 = line
                cap4 = ""
            elif re.match(r"^- ", line):
                cap4 = line

            # Loại bỏ các ký tự không mong muốn từ các cột cấp
            cap1 = re.sub(r"^[A-Z]\. ", "", cap1.strip())
            cap2 = re.sub(r"^\d+\. ", "", cap2.strip())
            cap3 = re.sub(r"^\d+\.\d+ ", "", cap3.strip())
            cap4 = re.sub(r"^- ", "", cap4.strip())

            # Thêm dòng mới vào danh sách data
            data.append([date_report, cap1, cap2, cap3, cap4, unit, day, month, year, note, kpi])

    # Chuyển danh sách data thành DataFrame và trả về kết quả
    result_df = pd.DataFrame(data, columns=["NGÀY BÁO CÁO", "CẤP 1", "CẤP 2", "CẤP 3", "CẤP 4", "ĐƠN VỊ", "NGÀY", "THÁNG", "NĂM", "GHI CHÚ", "CÔNG SUẤT THỰC TẾ (theo tích lũy năm)"])
    return result_df

class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        # Kiểm tra xem có phải là tệp mới được thêm vào không
        if not event.is_directory and (event.src_path.endswith('.xlsx') or event.src_path.endswith('.xls')):
            # Đường dẫn tới tệp mới
            new_file_path = event.src_path

            # Đọc ngày báo cáo từ dòng đầu tiên của file Excel mới nhất
            first_row_df = pd.read_excel(new_file_path, nrows=1)
            date_report = re.findall(r'\d{2}/\d{2}/\d{4}', first_row_df.columns[0])[0]
            date_report = datetime.datetime.strptime(date_report, '%d/%m/%Y').strftime('%Y-%m-%d')

            # Đọc dữ liệu từ file Excel mới nhất, bắt đầu từ dòng thứ 2 (index=1)
            df = pd.read_excel(new_file_path, skiprows=1)

            # Xử lý dữ liệu
            processed_df = process_data(df, date_report)
            print(processed_df)

            # Tạo tên file Excel mới
            output_file_name = 'Result_' + os.path.basename(new_file_path)

            # Tạo thư mục KẾT QUẢ nếu chưa tồn tại
            output_dir = os.path.expanduser('~/PVN_RESULT_DAILY/KET QUA')
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)

            # Định nghĩa đường dẫn đến file Excel mới
            output_file_path = os.path.join(output_dir, output_file_name)

            # Lưu kết quả vào file Excel mới
            processed_df.to_excel(output_file_path, index=False)

# Tạo một đối tượng giám sát
event_handler = MyHandler()
observer = Observer()

# Đường dẫn tới thư mục cần giám sát
directory_to_watch = os.path.expanduser('~/PVN_BÁO CÁO NGÀY/DATA_TEST')

# Đăng ký sự kiện giám sát và bắt đầu giám sát
observer.schedule(event_handler, path=directory_to_watch, recursive=True)
observer.start()

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    observer.stop()
observer.join()